In [17]:
import os
import numpy as np
import librosa
import pandas as pd

# 1. 读取手动标注副歌的文件
df_labels = pd.read_csv("chorus_labels.csv")  # eg: filename,start_sec,end_sec
label_dict = {}  # { filename -> [ (start,end), ... ] }

for _, row in df_labels.iterrows():
    fname = row["filename"]
    start_sec = float(row["start_sec"])
    end_sec = float(row["end_sec"])
    label_dict.setdefault(fname, []).append((start_sec, end_sec))

# 2. 遍历 raw audio 并分帧 => 提取特征 => 判断是否副歌 => 保存到 .npy
AUDIO_FOLDER = "../data/raw/"
FEATURE_FOLDER = "../data/processed/"   # 训练脚本中也用它
os.makedirs(FEATURE_FOLDER, exist_ok=True)

all_files = [f for f in os.listdir(AUDIO_FOLDER) if f.endswith(".mp3")]
metadata_list = []  # 用于存特征文件路径和标签

def extract_mfcc_mean(wave_data, sr, n_mfcc=13):
    # 示例特征：提取 MFCC 并对每帧均值 => shape(13,)
    mfcc = librosa.feature.mfcc(y=wave_data, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = mfcc.mean(axis=1)
    return mfcc_mean  # (13,)

frame_size = 2.0  # 每帧2秒
overlap_threshold = 0.5  # 帧与副歌区间交叠超过50%则记为副歌

for afile in all_files:
    path = os.path.join(AUDIO_FOLDER, afile)
    y, sr = librosa.load(path, sr=None)

    current_t = 0.0
    i = 0
    while True:
        start_sample = int(i * frame_size * sr)
        end_sample = int((i+1) * frame_size * sr)
        if start_sample >= len(y):
            break
        segment = y[start_sample:end_sample]

        # =========== 在此提取特征（例如MFCC均值） ===========
        feats = extract_mfcc_mean(segment, sr, n_mfcc=13)
        # ===============================================

        # 判定该帧是否副歌
        frame_start_sec = current_t
        frame_end_sec   = current_t + frame_size
        is_chorus = 0
        if afile in label_dict:
            for (s,e) in label_dict[afile]:
                # 计算交叠
                overlap = min(frame_end_sec,e) - max(frame_start_sec,s)
                if overlap > frame_size * overlap_threshold:
                    is_chorus = 1
                    break

        # 把特征 feats 保存为 .npy 文件
        frame_name = f"{afile}_{i}.npy"
        frame_path = os.path.join(FEATURE_FOLDER, frame_name)
        np.save(frame_path, feats)

        # 记录到 metadata_list
        # format: (feature_path, label)
        metadata_list.append((frame_path, is_chorus))

        current_t += frame_size
        i += 1

# 3. 生成 feature_index.csv 供 3_model_training.ipynb 使用
df_meta = pd.DataFrame(metadata_list, columns=["feature_path","label"])
df_meta.to_csv(os.path.join(FEATURE_FOLDER, "feature_index.csv"), index=False)
print("已生成特征文件 & feature_index.csv =>", os.path.join(FEATURE_FOLDER, "feature_index.csv"))

已生成特征文件 & feature_index.csv => ../data/processed/feature_index.csv
